# Long candle strategy  EDA

Long candle strategy is classification model.
Features: prices
Targets: -1,0,1 signal

To install conda and binance packages to this notebook uncomment the code below

In [1]:
%%capture
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
%pip install plotly==5.9.0
%pip install twisted
%pip install pandas
%pip install scikit-learn
#%pip install huobi-sdk==2.3.3

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [2]:
#mode="dev" # dev - work with local dev data
mode="yc" # yc - work with data from yandex cloud bots

symbol="BTC-USDT"
strategy="LongCandleDenseStrategy"

In [ ]:

import os

# Downloading latest pytrade2 data
if mode == "yc":
    os.system("cd ./../deploy/yandex_cloud; ./download_data.sh")
    data_dir=f"../data/yandex-cloud"
else:
    data_dir=f"../data/dev"
    
print(f"Download completed. Local data dir: {data_dir}")   

deploy_lib.sh: line 6: yc: command not found
INFO: Retrieving list of remote files for s3://pytrade2/data/ ...


Sync pytrade2 data from s3://pytrade2/data to /home/dima/projects/pytrade2/deploy/yandex_cloud/../../data/yandex-cloud


INFO: Cache file not found or empty, creating/populating it.
INFO: Compiling list of local files...
INFO: Running stat() and reading/calculating MD5 values on 24 files, this may take some time...
INFO: Found 28 remote files, 24 local files
INFO: Verifying attributes...
INFO: disabled md5 check for KerasBidAskRegressionStrategy/Xy/2023-12-16_BTC-USDT_raw_bid_ask.csv.zip
INFO: disabled md5 check for KerasBidAskRegressionStrategy/Xy/2023-12-17_BTC-USDT_raw_bid_ask.csv.zip
INFO: disabled md5 check for KerasBidAskRegressionStrategy/Xy/2023-12-17_BTC-USDT_x.csv.zip
INFO: Summary: 15 remote files to download, 0 local files to delete, 0 local files to hardlink


download: 's3://pytrade2/data/dev/KerasBidAskRegressionStrategy/KerasBidAskRegressionStrategy.db' -> '/home/dima/projects/pytrade2/deploy/yandex_cloud/../../data/yandex-cloud/dev/KerasBidAskRegressionStrategy/KerasBidAskRegressionStrategy.db'  [1 of 15]
 8192 of 8192   100% in    0s    56.78 KB/s  done
download: 's3://pytrade2/data/dev/KerasBidAskRegressionStrategy/Xy/2023-12-17_BTC-USDT_raw_bid_ask.csv.zip' -> '/home/dima/projects/pytrade2/deploy/yandex_cloud/../../data/yandex-cloud/dev/KerasBidAskRegressionStrategy/Xy/2023-12-17_BTC-USDT_raw_bid_ask.csv.zip'  [2 of 15]
 305 of 305   100% in    0s     3.91 KB/s  done
download: 's3://pytrade2/data/dev/KerasBidAskRegressionStrategy/Xy/2023-12-17_BTC-USDT_raw_candles_5min.csv.zip' -> '/home/dima/projects/pytrade2/deploy/yandex_cloud/../../data/yandex-cloud/dev/KerasBidAskRegressionStrategy/Xy/2023-12-17_BTC-USDT_raw_candles_5min.csv.zip'  [3 of 15]
 375 of 375   100% in    0s     4.04 KB/s  done
download: 's3://pytrade2/data/dev/KerasBid

In [ ]:
from pathlib import Path

def read_candles(days = 1, period = '1min'):
    """ Read last days' 1 min candles from file system """

    candles_dir = Path(data_dir, 'LongCandleDenseStrategy', 'Xy')
    files = sorted([f for f in os.listdir(candles_dir) if f.endswith(f"candles_{period}.csv.zip")])
    # Read last days' files to one dataframe
    df = pd.concat([pd.read_csv(Path(candles_dir, fname), parse_dates=['open_time', 'close_time']) for fname in files[-days:]])
    df = df.set_index('close_time', drop=False)
    del(df['close_time.1']) # temp fix
    # Resample because row data contains multiple candles inside a period
    return df.resample('1min').agg('last')

candles = read_candles(1)
candles.tail()



In [ ]:

def plot_value_counts(ax, df, col, grouped):
    signals = df[col]
    #vc = signals[signals.diff() != 0].value_counts()
    vc = df[col].value_counts() if not grouped else signals[(signals.diff() != 0) & (signals != 0)].value_counts()
    label_map={0:'oom', 1:'buy', -1: 'sell'}
    color_map={'oom':'C0', 'buy': 'C1', 'sell': 'C2'}
    labels = [ label_map[signal] for signal in vc.index.tolist()]
    colors = [color_map[key] for key in labels]
    ax.pie(vc, labels = labels,  autopct= lambda x: '{:.0f}'.format(x*vc.sum()/100), colors = colors)
    tag = 'groups' if grouped else ''
    ax.set_title(f"{col} {tag}")
    
def plot_signal_counts(df):
    fig, (ax1, ax2) = plt.subplots(1, 2) 
    plot_value_counts(ax1, df, 'signal', grouped = False)
    plot_value_counts(ax2, df, 'signal', grouped = True)
    fig.suptitle(f'Signal counts from {df.index.min()} to {df.index.max()}')
    plt.show()

In [ ]:

def calc_signal(candles, predict_window, open_ratio, min_profit_ratio, max_loss_ratio, comission_pct=0.012):
    """ Signal -1,0,1 plus extended debugging info """
    #next_candles = candles[['open', 'high', 'low', 'close', 'vol']].shift(1)

    next_candles = pd.DataFrame(index=candles.index)
    next_candles[['high', 'low']] = (candles[['high', 'low']][::-1]
                                     .rolling(predict_window).agg({'high' : 'max', 'low' : 'min'})
                                     [::-1])

    # BTC-USDT 40 000 * 1% = 400
    # BTC-USDT 40 000 * 0.012% = 40 * 0.012 = 4,8
    comission = comission_pct*0.01*2 # Order open, order close, double comission
    # Ratio to open: generate signal if profit/loss > open ratio
    #open_ratio = 1
    min_profit = candles['close']*(max(comission*2, min_profit_ratio))
    
    # Profit / loss > open ratio considering comission and minimal profit
    profit_buy = (next_candles['high'] - candles['high']) - (candles['close']*2*comission)
    loss_buy =  (candles['high'] - next_candles['low']) + (candles['close']*2*comission)
    signal_buy = (profit_buy > 0) & ((profit_buy / loss_buy) > open_ratio)& (profit_buy > min_profit)
    
    # Profit / loss > open ratio considering comission and minimal profit
    profit_sell = (candles['low'] - next_candles['low']) - (candles['close']*2*comission)
    loss_sell = (next_candles['high'] - candles['low']) + (candles['close']*2*comission)
    signal_sell = (profit_sell > 0) & ((profit_sell / loss_sell) > open_ratio) & (profit_sell > min_profit)

    # Signal
    signal = pd.DataFrame(index=candles.index)
    signal['signal'] = 0  # Default to 0
    signal.loc[signal_buy & ~signal_sell, 'signal'] = 1  # Set to 1 where 'buy' is True and 'sell' is False
    signal.loc[~signal_buy & signal_sell, 'signal'] = -1  # Set to -1 where 'sell' is True and 'buy' is False

    df = signal
    # Expected profit and loss
    df.loc[signal['signal']==1, 'profit'] = profit_buy
    df.loc[signal['signal']==1, 'loss'] = loss_buy
    df.loc[signal['signal']==-1, 'profit'] = profit_sell
    df.loc[signal['signal']==-1, 'loss'] = loss_sell
    return df
    # df[['high', 'low', 'close']] = candles[['high','low', 'close']]
    # df[['next_high', 'next_low']] = next_candles[['high', 'low']]
    # df['min_profit'] = min_profit
    # df['profit_buy'] = profit_buy
    # df['loss_buy']  = loss_buy
    # df['profit_buy_ratio'] = profit_buy / loss_buy
    # df['profit_sell'] = profit_sell
    # df['loss_sell']  = loss_sell
    # df['profit_sell_ratio'] = profit_sell / loss_sell

    # #df = df[(profit_buy / loss_buy) > open_ratio]
    
    # df['signal'] = signal['signal']
    # # df['buy ratio'] = profit_buy/loss_buy
    
    # return df

signal = (calc_signal(candles, 
                      predict_window = '10min', 
                      open_ratio = 4, 
                      # 0.01*0.012*2 - open+close comission 0.012%
                      # Comission is 4.8 for 40K price, ~10 for one open+close trade
                      min_profit_ratio = (0.01*0.012*2) *2,
                      max_loss_ratio = (0.01*0.012*2) * 10)
         )
signal[signal['signal']!=0].tail(100)

plot_signal_counts(signal)



## Account

In [ ]:
import plotly.express as px
import seaborn as sns
from matplotlib import pyplot as plt
import seaborn as sns

plot_figsize=(10,5)
sns.set_style("darkgrid")
from datetime import datetime


def get_balance(strategy: str):
    """ Read balance history from data"""
    
    data_dir=f"../data"
    env_dir="yandex-cloud"
    account_dir=f"{data_dir}/{env_dir}/{strategy}/account"
    print(f"Reading balance files from {account_dir}")
    #file_path = sorted([f for f in os.listdir(account_dir) if f.endswith(f"{symbol}_{name}.csv")])[-1]
    files = sorted([f for f in os.listdir(account_dir) if f.endswith(f".csv")])
    dfs = [pd.read_csv(f"{account_dir}/{f}", parse_dates=True, index_col="time") for f in files]
    df = pd.concat(dfs).sort_index() if dfs else pd.DataFrame(data=[{"datetime": datetime.now(), "balance": 0}], columns=["datetime","balance"])
    #df = pd.concat([df for df in dfs if not df.empty]).sort_index()
    df["datetime"] = df.index
    return df

balance = get_balance(strategy)
balance["balance"].plot()
plt.show()
px.line(balance, "datetime", "balance", title=f"{strategy} balance USDT").update_traces(mode='lines+markers').show()
plt.show()

## Predicted and actual low/high

In [ ]:
import os
import glob
import pandas as pd
from datetime import timedelta,date
import matplotlib.pyplot as plt
import plotly.express as px
import logging

def read_last(symbol: str, name: str, n=1000):
    xy_dir=f"{data_dir}/{strategy}/Xy"
    file_path = sorted([f for f in os.listdir(xy_dir) if f.endswith(f"{symbol}_{name}.csv.zip")])[-1]
    file_path = f"{xy_dir}/{file_path}"
    print(f"Read from {file_path}")
    return pd.read_csv(file_path, parse_dates=True).tail(n)


def read_last_xy(data_dir):
    """ Read days from start to end from local predictions history"""
    
    #data=read_last(symbol, "data", n)
    X=read_last(symbol, "x").set_index('close_time')
    y=read_last(symbol, "y").set_index('close_time')
    data=pd.merge(X,y, left_index=True, right_index=True) 
    #y=pd.concat([data,y])
    return data,X,y


# Read candles for last 3 days
print(symbol)
data,x,y = read_last_xy(data_dir)
y.tail()


In [ ]:

#actual_signal_counts = y['signal_actual'].diff().to_numpy().nonzero().value_counts()
actual_signal_counts = y['signal_actual'].value_counts()


#actual_signal_counts = actual_signal_counts[actual_signal_counts['signal_actual']!=0]
pred_signal_counts = y['signal_pred'].value_counts()
print(actual_signal_counts)


In [ ]:


def plot_signal_actual_pred_counts(y):
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)        
    
    # def plot_value_counts(ax, df, col, grouped=False):
    #     signals = df[col]
    #     #vc = signals[signals.diff() != 0].value_counts()
    #     vc = df[col].value_counts() if not grouped else signals[(signals.diff() != 0) & (signals != 0)].value_counts()
    #     label_map={0:'oom', 1:'buy', -1: 'sell'}
    #     color_map={'oom':'C0', 'buy': 'C1', 'sell': 'C2'}
    #     labels = [ label_map[signal] for signal in vc.index.tolist()]
    #     colors = [color_map[key] for key in labels]
    #     ax.pie(vc, labels = labels,  autopct= lambda x: '{:.0f}'.format(x*vc.sum()/100), colors = colors)
    #     tag = 'groups' if grouped else ''
    #     ax.set_title(f"{col} {tag}")
    
    
    plot_value_counts(ax1, y, 'signal_actual', grouped=False)
    plot_value_counts(ax2, y, 'signal_pred', grouped=False)
    
    plot_value_counts(ax3, y, 'signal_actual', grouped=True)
    plot_value_counts(ax4, y, 'signal_pred', grouped=True)
    fig.suptitle('Signal counts')
    plt.show()

plot_signal_actual_pred_counts(y)    

In [ ]:
import seaborn as sns

sns.heatmap(y.corr(), annot=True)
plt.title('Predicted and actual signal')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

clf_report = classification_report(y['signal_actual'],
                                   y['signal_pred'],
                                   zero_division=0,
                                   #labels=['sell', 'flat', 'buy'],
                                   target_names=['sell', 'flat', 'buy'],
                                   output_dict=True)
#print(clf_report)
#sns.heatmap(pd.DataFrame(clf_report).iloc[:-1, :].T, annot=True)
#sns.heatmap(pd.DataFrame(clf_report).iloc[:-1, :-3].T[::-1], annot=True)
sns.heatmap(pd.DataFrame(clf_report).iloc[:-1, :].T, annot=True)
plt.show()

In [ ]:
from sklearn import metrics
confusion_matrix = metrics.confusion_matrix(y['signal_actual'], y['signal_pred'])

metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ['Sell', 'Flat', 'Buy']).plot()
plt.title('Confusion matrix')
plt.show()
print(confusion_matrix)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.plot(y.index, y['signal_pred']-y['signal_actual'])
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=5))
plt.xticks(rotation=45)

#ax.format_xdata = mdates.DateFormatter('%Y-%m-%d')
#plt.gca().xaxis.set_major_formatter(ConsizeDateFormatter())
plt.show()

In [ ]:
import plotly.express as px
px.bar(x=y.index, 
    y=y['signal_pred']-y['signal_actual'], 
    title='Predicted and actual signal difference. diff = predicted - actual',
    labels = {'x': 'time', 'y':'predicted - actual'}
      ).show()